Скрипт сделал pawsfpv

# Подготовка llama.cpp

In [ ]:
# @title  {"display-mode":"form"}

from google.colab import drive

#@markdown ___
#@markdown ## Собрать llama.cpp из исходников
#@markdown Сохранит программу на гугл диск и будет использовать её для быстрого включения.
#@markdown Первое включение очень долгое, сборка занимает до получаса
build = False #@param {type:"boolean"}
#@markdown ___
#@markdown ## Вариант сборки llama.cpp (cpu или cuda)
#@markdown Для CUDA сменить среду выполнения на T4, иначе сменить на CPU.

#@markdown Важно правильно переключить среду выполнения для первых включений в соответствии с галочкой ниже, при неправильной среде выполнения потребуется пересборка программы
cuda = False # @param {"type":"boolean"}
#@markdown Сохранение всего набора программ
save_full = False #@param {type:"boolean"}
#@markdown ___
#@markdown ## Пересборка llama.cpp
#@markdown Это удалит уже собранную llama.cpp с гугл диска, если она была. Будет повторная сборка, как при первом включении
rebuild = False #@param {type:"boolean"}

#@markdown ___
#@markdown #Или скачать готовую по ссылке (когда build выключено)
link_ready = "https://github.com/p-fpv/llamacpp_colab/releases/download/ver1/llamaCUDA.zip" #@param {type:"string"}


!rm -rf ./build
!rm -rf ./llamaCUDA.zip
!rm -rf ./llamaCPU.zip

if build == 1:
  drive.mount('/content/drive')
  import os.path

  if cuda:
    if rebuild:
      !rm -rf /content/drive/MyDrive/llamaCUDA.zip

    file_path1 = "/content/drive/MyDrive/llamaCUDA.zip"
    if os.path.exists(file_path1):
      !rsync -arh --info=progress2 /content/drive/MyDrive/llamaCUDA.zip /content/llamaCUDA.zip
      !7z x llama*.zip -y
    else:
      !rm -rf /content/llama.cpp
      !apt install ccache
      !git clone https://github.com/ggml-org/llama.cpp
      #cuda
      !cd /content/llama.cpp/;cmake -B build -DGGML_CUDA=ON -DBUILD_SHARED_LIBS=OFF;cmake --build build --config Release -j 2
      !mkdir /content/build/
      if save_full:
        !rsync -arh --info=progress2 /content/llama.cpp/build/bin /content/build/
      else:
        !mkdir /content/build/bin/
        !rsync -arh --info=progress2 /content/llama.cpp/build/bin/llama-server /content/build/bin/
      !7z a llamaCUDA.zip /content/build
      !rsync -arh --info=progress2 /content/llamaCUDA.zip /content/drive/MyDrive/llamaCUDA.zip

  else:
    if rebuild:
      !rm -rf /content/drive/MyDrive/llamaCPU.zip

    file_path2 = "/content/drive/MyDrive/llamaCPU.zip"
    if os.path.exists(file_path2):
      !rsync -arh --info=progress2 /content/drive/MyDrive/llamaCPU.zip /content/llamaCPU.zip
      !7z x llama*.zip -y
    else:
      !rm -rf /content/llama.cpp
      !git clone https://github.com/ggml-org/llama.cpp
      #CPU
      !cd /content/llama.cpp/;cmake -B build -DGGML_CUDA=OFF -DBUILD_SHARED_LIBS=OFF;cmake --build build --config Release -j 2
      !mkdir /content/build/
      if save_full:
        !rsync -arh --info=progress2 /content/llama.cpp/build/bin /content/build/
      else:
        !mkdir /content/build/bin/
        !rsync -arh --info=progress2 /content/llama.cpp/build/bin/llama-server /content/build/bin/
      !7z a llamaCPU.zip /content/build
      !rsync -arh  --info=progress2 /content/llamaCPU.zip /content/drive/MyDrive/llamaCPU.zip

elif build == 0:
  !rm -rf ./build
  !wget $link_ready
  !7z x llama*.zip

from IPython.display import clear_output
clear_output()
print("Ok")
#@markdown ---

# Model Download

In [ ]:
# @title  {"display-mode":"form"}
#@markdown  ___
use_model_link = "2" # @param ["1", "2", "3", "4", "5"]
#@markdown  ___

link1 = "https://huggingface.co/IlyaGusev/saiga_yandexgpt_8b_gguf/resolve/main/saiga_yandexgpt_8b.Q8_0.gguf" #@param {type:"string"}
link2 = "https://huggingface.co/IlyaGusev/saiga_yandexgpt_8b_gguf/resolve/main/saiga_yandexgpt_8b.Q6_K.gguf" #@param {type:"string"}
link3 = "https://huggingface.co/bartowski/gemma-2-2b-it-GGUF/resolve/main/gemma-2-2b-it-Q6_K_L.gguf" #@param {type:"string"}
link4 = "https://huggingface.co/bartowski/gemma-2-2b-it-GGUF/resolve/main/gemma-2-2b-it-Q4_K_M.gguf" #@param {type:"string"}
link5 = "https://huggingface.co/Qwen/Qwen2.5-Coder-7B-Instruct-GGUF/resolve/main/qwen2.5-coder-7b-instruct-q8_0.gguf" #@param {type:"string"}
#@markdown  ___
if use_model_link == "1":
  link = link1
elif use_model_link == "2":
  link = link2
elif use_model_link == "3":
  link = link3
elif use_model_link == "4":
  link = link4
elif use_model_link == "5":
  link = link5
print(link)
!rm -rf ./*.gguf
!wget $link

# https://huggingface.co/IlyaGusev/saiga_yandexgpt_8b_gguf
from IPython.display import clear_output
clear_output()
print("Ok")

# Включить llama-server

In [ ]:
# @title  {"display-mode":"form"}
!pip install pyngrok
from IPython.display import clear_output
clear_output()

import getpass
from pyngrok import ngrok, conf
ngrok.kill()
!sleep 1
#@markdown ___
Ngrok_token = "" #@param {type:"string"}
#@markdown  https://dashboard.ngrok.com/get-started/your-authtoken

#@markdown ___

#srv=ngrok.connect(7860, pyngrok_config=conf.PyngrokConfig(auth_token="") , bind_tls=True).public_url
port = "5000"

# conf.get_default().auth_token = getpass.getpass()
public_url = ngrok.connect(port, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token)).public_url
# print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
print(f" * ngrok tunnel \"{public_url}\"")

cnt = 32768 #@param {type:"number"}
ngl = 100
# apikey = "" #@param {type:"string"}

#@markdown  ___
logs = False #@param {type:"boolean"}

if logs:
  !cd /content/build/bin/; chmod +x ./*; ./llama-server \
  -m /content/*.gguf \
  --host 0.0.0.0 --port $port \
  -c $cnt -fa -t 2 --mlock --no-mmap -ngl $ngl --parallel 1 #--api-key apikey # --parallel 1 --no-warmup
else:
  !cd /content/build/bin/; chmod +x ./*; ./llama-server \
  -m /content/*.gguf \
  --host 0.0.0.0 --port $port \
  -c $cnt -fa -t 2 --mlock --no-mmap -ngl $ngl --parallel 1 2>/dev/null

# from IPython.display import clear_output
logs_cls = True #@param {type:"boolean"}
#@markdown  ___
if logs_cls:
  clear_output()
print("Ok")